In [3]:
import numpy as np

In [4]:
def proj(matrix):
    return np.kron(matrix, matrix.conj().T)

In [5]:
qubit0 = np.array([[1], [0]])
qubit1 = np.array([[0], [1]])

qubit = [qubit0, qubit1]

In [6]:
stan0 = np.kron(qubit[0], qubit[0])
stan1 = np.kron(qubit[1], qubit[0])
stan2 = np.kron(qubit[0], qubit[1])

In [7]:
stan0

array([[1],
       [0],
       [0],
       [0]])

In [8]:
stan0teo = np.array([['a0'], ['b0'], ['c0'], ['d0']])
stan1teo = np.array([['a1'], ['b1'], ['c1'], ['d1']])
stan2teo = np.array([['a2'], ['b2'], ['c2'], ['d2']])

In [9]:
baza = [stan0, stan1, stan2]
bazateo = [stan0teo, stan1teo, stan2teo]

In [7]:
check = proj(baza[0]) + proj(baza[1]) + proj(baza[2])
checkteo = proj(bazateo[0]) + proj(bazateo[1]) + proj(bazateo[2])

C:\Users\szymon\AppData\Local\Temp\ipykernel_17504\4267056448.py:2: DeprecationWarning: attempting to conjugate non-numeric dtype; this will error in the future to match the behavior of np.conjugate
  return np.kron(matrix, matrix.conj().T)


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U2'), dtype('<U2')) -> None

In [9]:
np.kron(qubit[0], qubit[1].conj().T)

array([[0, 1],
       [0, 0]])

In [8]:
np.kron(qubit[1], qubit[0].conj().T)

array([[0, 0],
       [1, 0]])

In [19]:
import numpy as np
from scipy.linalg import eig
from numpy.linalg import norm
# Definicja prymitywnego pierwiastka z jedności
omega = np.exp(2 * np.pi * 1j / 3)

# Definicja bramki F w bazie |00>, |01>, |10>, |11>
F = 1/np.sqrt(3) * np.array([
    [1,      1,      1,      0],
    [1, omega,      omega**2,      0],
    [1,      omega**2, omega,    0],
    [0,      0,      0,      1]
], dtype=complex)

# Obliczanie wartości własnych i wektorów własnych operatora F
wartosci_wlasne, wektory_wlasne = eig(F)

# Wyświetlenie wartości własnych
print("Wartości własne operatora F:")
print(wartosci_wlasne)

# Wyświetlenie wektorów własnych
print("\nWektory własne operatora F:")
for i in range(len(wektory_wlasne)):
    print(f"Wektor własny {i}: {wektory_wlasne[:, i]}")

# Wybór pierwszych trzech wektorów własnych jako zakodowane stany
v0 = wektory_wlasne[:, 0]
v1 = wektory_wlasne[:, 1]
v2 = wektory_wlasne[:, 2]

# Ortonormalizacja wektorów
def gram_schmidt(vectors):
    ortho_vectors = []
    for v in vectors:
        for ortho_v in ortho_vectors:
            v = v - np.dot(np.conj(ortho_v), v) * ortho_v
        v = v / norm(v)
        ortho_vectors.append(v)
    return np.array(ortho_vectors)

wektory_ortonormalne = gram_schmidt([v0, v1, v2])
v0o = wektory_ortonormalne[0]
v1o = wektory_ortonormalne[1]
v2o = wektory_ortonormalne[2]

# Sprawdzenie ortonormalności
print("\nSprawdzenie ortonormalności:")
print(f"Norma v0o: {norm(v0o)}")
print(f"Norma v1o: {norm(v1o)}")
print(f"Norma v2o: {norm(v2o)}")
print(f"Iloczyn skalarny v0o i v1o: {np.dot(np.conj(v0o), v1o)}")
print(f"Iloczyn skalarny v0o i v2o: {np.dot(np.conj(v0o), v2o)}")
print(f"Iloczyn skalarny v1o i v2o: {np.dot(np.conj(v1o), v2o)}")

# Sprawdzenie działania operatora F na wektorach
Fv0 = F @ v0o
Fv1 = F @ v1o
Fv2 = F @ v2o

wartosc_wlasna_v0 = np.vdot(v0o, Fv0)
wartosc_wlasna_v1 = np.vdot(v1o, Fv1)
wartosc_wlasna_v2 = np.vdot(v2o, Fv2)

print("\nWartości własne po ortonormalizacji:")
print(f"Wartość własna dla v0o: {wartosc_wlasna_v0}")
print(f"Wartość własna dla v1o: {wartosc_wlasna_v1}")
print(f"Wartość własna dla v2o: {wartosc_wlasna_v2}")

# Definicja operatora projekcji
def projector(v):
    return np.outer(v, np.conj(v))

# Inicjalizacja bramki CZ jako macierzy zerowej 16x16
CZ = np.zeros((16, 16), dtype=complex)

# Definicja identyczności 4x4
I4 = np.identity(4, dtype=complex)

# Tworzenie bramki CZ
for i in range(3):
    temp_projector = projector(wektory_ortonormalne[i])
    if np.allclose(wartosci_wlasne[i], omega):
        temp_matrix = np.kron(temp_projector, F)
    elif np.allclose(wartosci_wlasne[i], omega**2):
        temp_matrix = np.kron(temp_projector, np.linalg.matrix_power(F, 2))
    else:
        temp_matrix = np.kron(temp_projector, I4)
    CZ += temp_matrix

# Sprawdzenie wymiarów bramki CZ
print("\nWymiary bramki CZ:", CZ.shape)

# Definicja stanu
stan = np.kron(v0o, v1o)
print("Wymiary stanu:", stan.shape)

# Mnożenie bramki CZ przez stan
CZ_stan = CZ @ stan

# Wyświetlenie wyniku
print("\nStan po zastosowaniu bramki CZ:")
print(CZ_stan)

# Sprawdzenie czy wynik jest w podprzestrzeni kodowania
def is_in_subspace(state, basis_vectors):
    projections = [np.vdot(bv, state) for bv in basis_vectors]
    reconstructed_state = sum(proj * bv for proj, bv in zip(projections, basis_vectors))
    return np.allclose(state, reconstructed_state)

in_subspace = is_in_subspace(CZ_stan, [np.kron(v0o, v0o), np.kron(v1o, v1o), np.kron(v2o, v2o)])

print("\nCzy wynikowy stan jest w podprzestrzeni kodowania?", in_subspace)

Wartości własne operatora F:
[ 1.00000000e+00+5.80649261e-17j -1.00000000e+00+2.16701254e-16j
  2.22044605e-16+1.00000000e+00j  5.77350269e-01+0.00000000e+00j]

Wektory własne operatora F:
Wektor własny 0: [0.88807383+0.00000000e+00j 0.32505758+4.46574153e-17j
 0.32505758+4.46574153e-17j 0.        -0.00000000e+00j]
Wektor własny 1: [-0.45970084-6.31551224e-17j  0.62796303+0.00000000e+00j
  0.62796303+0.00000000e+00j  0.        +0.00000000e+00j]
Wektor własny 2: [ 1.60246891e-17-2.04776892e-35j -7.07106781e-01+2.77555756e-17j
  7.07106781e-01+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
Wektor własny 3: [0.+0.j 0.+0.j 0.+0.j 1.+0.j]

Sprawdzenie ortonormalności:
Norma v0o: 1.0
Norma v1o: 1.0
Norma v2o: 1.0
Iloczyn skalarny v0o i v1o: (3.2566533477189495e-33+0j)
Iloczyn skalarny v0o i v2o: (2.7755575615628914e-17-2.311115933264683e-33j)
Iloczyn skalarny v1o i v2o: (5.551115123125783e-17+0j)

Wartości własne po ortonormalizacji:
Wartość własna dla v0o: (1.0000000000000002+3.6088641372

In [21]:
wektory_wlasne[0].round(4)

array([ 0.8881+0.j, -0.4597-0.j,  0.    -0.j,  0.    +0.j])

In [22]:
wektory_wlasne[1].round(4)

array([ 0.3251+0.j,  0.628 +0.j, -0.7071+0.j,  0.    +0.j])

In [23]:
wektory_wlasne[2].round(4)

array([0.3251+0.j, 0.628 +0.j, 0.7071+0.j, 0.    +0.j])

In [24]:
wektory_wlasne[3].round(4)

array([0.-0.j, 0.+0.j, 0.+0.j, 1.+0.j])

In [15]:
bellstatePhiMin = 1/np.sqrt(2) * (np.kron(qubit[0], qubit[1]) - np.kron(qubit[1], qubit[0]))

In [17]:
bellstatePhiPlus = 1/np.sqrt(2) * (np.kron(qubit[0], qubit[1]) + np.kron(qubit[1], qubit[0]))

In [18]:
bellstatePhiPlus

array([[0.        ],
       [0.70710678],
       [0.70710678],
       [0.        ]])

In [25]:
import numpy as np
from scipy.linalg import eig
from numpy.linalg import norm

omega = np.exp(2 * np.pi * 1j / 3)

# Zredukowana macierz F o wymiarze 3x3
F_reduced = (1 / np.sqrt(3)) * np.array([
    [1,          1,          1],
    [1,      omega,      omega**2],
    [1,  omega**2,      omega]
], dtype=complex)

# Obliczanie wartości własnych i wektorów własnych
wartosci_wlasne, wektory_wlasne = eig(F_reduced)

print("Wartości własne operatora F_reduced:")
for i, wartosc in enumerate(wartosci_wlasne):
    print(f"Wartość własna {i}: {wartosc}")

print("\nWektory własne operatora F_reduced:")
for i in range(len(wektory_wlasne)):
    print(f"Wektor własny {i}: {wektory_wlasne[:, i]}")

# Ortonormalizacja wektorów
def gram_schmidt(vectors):
    ortho_vectors = []
    for v in vectors:
        for u in ortho_vectors:
            v = v - np.dot(u.conj().T, v) * u
        norm_v = np.linalg.norm(v)
        if norm_v > 1e-10:
            v = v / norm_v
            ortho_vectors.append(v)
    return np.array(ortho_vectors)

wektory_ortonormalne = gram_schmidt(wektory_wlasne.T)
v0o = wektory_ortonormalne[0]
v1o = wektory_ortonormalne[1]
v2o = wektory_ortonormalne[2]

# Sprawdzenie ortonormalności
print("\nSprawdzenie ortonormalności:")
print(f"Norma v0o: {norm(v0o)}")
print(f"Norma v1o: {norm(v1o)}")
print(f"Norma v2o: {norm(v2o)}")
print(f"Iloczyn skalarny v0o i v1o: {np.dot(np.conj(v0o), v1o)}")
print(f"Iloczyn skalarny v0o i v2o: {np.dot(np.conj(v0o), v2o)}")
print(f"Iloczyn skalarny v1o i v2o: {np.dot(np.conj(v1o), v2o)}")


Wartości własne operatora F_reduced:
Wartość własna 0: (1.0000000000000004+5.80649260633834e-17j)
Wartość własna 1: (-1+2.1670125420627727e-16j)
Wartość własna 2: (2.220446049250313e-16+1j)

Wektory własne operatora F_reduced:
Wektor własny 0: [0.88807383+0.00000000e+00j 0.32505758+4.46574153e-17j
 0.32505758+4.46574153e-17j]
Wektor własny 1: [-0.45970084-6.31551224e-17j  0.62796303+0.00000000e+00j
  0.62796303+0.00000000e+00j]
Wektor własny 2: [ 1.60246891e-17-2.04776892e-35j -7.07106781e-01+2.77555756e-17j
  7.07106781e-01+0.00000000e+00j]

Sprawdzenie ortonormalności:
Norma v0o: 1.0
Norma v1o: 1.0
Norma v2o: 1.0
Iloczyn skalarny v0o i v1o: (3.2566533477189495e-33+0j)
Iloczyn skalarny v0o i v2o: (2.7755575615628914e-17-2.311115933264683e-33j)
Iloczyn skalarny v1o i v2o: (5.551115123125783e-17+0j)


In [27]:
wektory_wlasne[0].round(3)

array([ 0.888+0.j, -0.46 -0.j,  0.   -0.j])

In [28]:
wektory_wlasne[1].round(3)

array([ 0.325+0.j,  0.628+0.j, -0.707+0.j])

In [29]:
wektory_wlasne[2].round(3)

array([0.325+0.j, 0.628+0.j, 0.707+0.j])